# Test to generate document from existing structure

[x] Define structure as Pydantic object

[ ] Define subclasses for tools, facts and sections

[ ] Tools

[ ] Direct generation



Tools - extract information form PDF is pretty brittle, let's do tools directly
Gemini multimodel - 

In [15]:
from typing import List, Literal

from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

from langchain_google_vertexai import ChatVertexAI

In [67]:
llm = ChatVertexAI(model_name='gemini-1.5-pro-001', temperature=0)

In [77]:
class Section(BaseModel):
    name: str
    description: str

class Report(BaseModel):
    name: str = Field(description='Name of the report', required=True)
    description: str = Field(description='Description of the report, this field is used to search the right structure for the request')
    tools: List[str]
    facts: List[str]
    sections: List[Section]

In [79]:
stock_report = Report(
    name='Stocks',
    description='Simple report on last',
    tools = ['get_info'],
    facts = ['revenues'],
    sections = [
        Section(name = 'Revenue', description='Comparision of revenue between current and previous quarters and the same quarter of current and last year')
    ]
)

In [70]:
@tool
def get_info(kpi: str, q: Literal['Q12024', 'Q22024', 'Q12023', 'Q22023']):
    """
    Get Financial information about Alphabet
    """
    data = {
        'Q12024': 80539,
        'Q12023': 69787,
        'Q22024': 84742,
        'Q22023': 74604
    }

    return f'{data[q]}'

In [71]:
llm_tools = llm.bind_tools(tools = [get_info])

In [72]:
response1 = llm_tools.invoke('Compare Alphabet revenue from Q2 this year to last year')

In [73]:
messages.append(response1)

for tool_call in response1.tool_calls:
    selected_tool = {"get_info": get_info}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [74]:
response2 = llm_tools.invoke(messages)

### Using KPI to create response

Pure function calling responses does create challenges - already prepared KPIs (from another sections), format, etc

Another possible aproach - create XML with all KPIs as text and then request model to generate text

Example:
```
<REVENUE UNIT="milion $">
	<Q22024>110</Q22024>
	<Q22023>100</Q22023>
<REVENUE>

```

potential way to work with xml - xmltodict

In [75]:
response2

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_info', 'arguments': '{"kpi": "Operating Income", "q": "Q22024"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_LOW'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 203, 'candidates_token_count': 13, 'total_token_count': 216}, 'finish_reason': 'STOP'}, id='run-7661af62-d4ec-4d4b-ad70-bcdd4ee7240d-0', tool_calls=[{'name': 'get_info', 'args': {'kpi': 'Operating Income', 'q': 

# Gemini multimodel with PDF on local disk

```
with open('2024q1-alphabet-earnings-release-pdf.pdf', 'rb') as f:
    data = f.read()


pdf_message = {
    "type": "media",
    "mime_type": "application/pdf",
    "data": data,
}

text_message = {
    "type": "text",
    "text": "What was Operating income in Q1 2024 and on which page it is mentioned?",
}

messages = HumanMessage(content=[pdf_message, text_message])
```

In [47]:
@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

query = "What is 3 * 12? Also, what is 11 + 49?"

tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

print(messages)

response = llm.invoke(messages)

This model can reply with multiple function calls in one response. Please don't rely on `additional_kwargs.function_call` as only the last one will be saved.Use `tool_calls` instead.


[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 74, 'candidates_token_count': 6, 'total_token_count': 80}, 'finish_reason': 'STOP'}, id='run-2a6286b8-f8c3-4f63-b647-c0c4390430ac-0', tool_calls=[{'name': 'multiply

In [49]:
messages[3]

ToolMessage(content='60', tool_call_id='c3c9ac10-763e-4969-982a-90a316c98b4e')